In [1]:
import csv
from collections import Counter
import pandas as pd
from itertools import chain, combinations
from tabulate import tabulate
import time

In [2]:
# Read the .pb file 

path = "netherlands_amsterdam_166_.pb"  # set this variable
with open(path, 'r', newline='', encoding="utf-8") as csvfile:
    meta = {}
    projects = {}
    votes = {}
    section = ""
    header = []
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        if str(row[0]).strip().lower() in ["meta", "projects", "votes"]:
            section = str(row[0]).strip().lower()
            header = next(reader)
        elif section == "meta":
            meta[row[0]] = row[1].strip()
        elif section == "projects":
            projects[row[0]] = {}
            for it, key in enumerate(header[1:]):
                projects[row[0]][key.strip()] = row[it+1].strip()
        elif section == "votes":
            votes[row[0]] = {}
            for it, key in enumerate(header[1:]):
                votes[row[0]][key.strip()] = row[it+1].strip()

In [3]:
#Extract required data from the file
C=set(projects.keys())
V=set(votes.keys())
n=len(V)
approval_dict = {key: set(value['vote'].split(',')) for key, value in votes.items()}

In [4]:
lengths = [len(value) for value in approval_dict.values()]

# Calculate the average length
average_length = sum(lengths) / len(lengths)

print("Average length of values:", average_length)

Average length of values: 11.927230046948356


In [5]:
max_length = min(len(value) for value in approval_dict.values())
max_length

1

In [6]:
C

{'12416',
 '12417',
 '12418',
 '12419',
 '12420',
 '12421',
 '12422',
 '12423',
 '12424',
 '12425',
 '12426',
 '12427',
 '12428',
 '12429',
 '12430',
 '12431',
 '12432',
 '12433',
 '12434',
 '12435',
 '12436',
 '12437',
 '12438',
 '12439',
 '12440',
 '12441',
 '12442',
 '12443',
 '12444',
 '12445',
 '12446',
 '12447',
 '12448',
 '12449',
 '12450',
 '12451',
 '12452',
 '12453',
 '12454',
 '12455',
 '12456',
 '12457',
 '12458',
 '12459',
 '12460',
 '12461',
 '12462',
 '12463',
 '12464',
 '12465',
 '12466',
 '12467'}

In [7]:
#Creating a df with the projects and the number of votes they got
all_values = set()
for value_set in approval_dict.values():
    all_values.update(value_set)

# Count occurrences of each value in the entire dictionary
count_dict = Counter()
for value_set in approval_dict.values():
    count_dict.update(value_set)

# Create a list with unique values and their counts
result_list = [[value, count_dict[value]] for value in all_values]

greedy_list = pd.DataFrame(result_list, columns=['Value', 'Count'])
greedy_list = greedy_list.sort_values(by='Count', ascending=False)

In [8]:
greedy_list.head(3)

,Value,Count
11,12437,242
36,12431,205
50,12422,167


In [9]:
votes

{'16618473904': {'vote': '12416,12421,12422,12424,12425,12426,12427,12430,12435,12437,12438,12442,12445,12447,12449,12455,12456,12462,12463'},
 '19885305232': {'vote': '12416,12423,12426,12427,12430,12438,12439,12446,12451,12459,12463,12465'},
 '11989741802': {'vote': '12416,12419,12420,12421,12422,12424,12426,12428,12430,12431,12433,12434,12435,12436,12437,12439,12441,12442,12446,12451,12453,12454,12455,12456,12457'},
 '13023775231': {'vote': '12416,12419,12421,12422,12423,12424,12426,12430,12431,12432,12433,12435,12437,12438,12439,12442,12443,12447,12449,12452,12454,12455,12457,12462,12463,12464,12466,12467'},
 '15917937648': {'vote': '12416,12431,12436,12446,12450,12453,12461'},
 '12446094623': {'vote': '12416,12418,12421,12422,12424,12426,12430,12431,12432,12433,12434,12435,12439,12443,12445,12447,12449,12452,12455,12461,12464,12465,12466,12467'},
 '12540032756': {'vote': '12416,12420,12424,12425,12426,12428,12430,12431,12432,12433,12434,12435,12437,12438,12439,12440,12443,12444,12

In [ ]:

def powerset(iterable, max_size): 
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, min(len(s)+1, max_size+1)))

# Define the function to create the set W dynamically based on its size
def create_set_W(df, W):
    return set(df.head(W)['Value'])
# Define the function for JR
def JR(non_winners, approval_dict, W_set, k, n):
    start_time = time.time()  # Record start time
    counts = {}
    breaks_jr = False

    for non_winner in non_winners:
        counts[non_winner] = 0
        for key, value in approval_dict.items():
            if non_winner in value and not set(W_set).intersection(value):
                counts[non_winner] += 1
                if counts[non_winner] >= n / k:
                    breaks_jr = True
                    break
        if breaks_jr:
            return "Breaks JR."
        
    end_time = time.time()  # Record end time
    duration = end_time - start_time  # Calculate duration
    return "Satisfies JR.", duration

# Define the function for EJR
def EJR(approval_lists, winners, k, n, C,l):
    start_time = time.time()  # Record start time
    
    #FOR L in 1 to k+1, 
    #new C= where C have atleast nl/k votes 
    C_modified = [candidate for candidate in C if greedy_list.loc[greedy_list['Value'] == candidate, 'Count'].iloc[0] >= l * n / k]

    # Generate subsets S using C_modified
    S = set(s for s in powerset(C_modified, k) if not set(s).issubset(winners)) #create all subsets of size <=k
    #S = set(s for s in powerset(C, k) if not set(s).issubset(winners)) #create all subsets of size <=k

    for s in S:#iterate through all subsets
        l = len(s)
        count = 0
        for approval_list in approval_lists.values(): #iterate through all approval_lists/profile
            if set(s).issubset(approval_list): #if s is a subset of the profile
                if abs(len(set(approval_list) & set(winners))) < l: #if profile intersection W is < l
                    count += 1
        if count >= l * n / k:
            end_time = time.time()  # Record end time
            duration = end_time - start_time  # Calculate duration
            return "breaks EJR", duration
            
    end_time = time.time()  # Record end time
    duration = end_time - start_time  # Calculate duration
    return "Satisfies EJR", duration

W_range = range(1, 11)
results = []

# Iterate over different values of W
for l in W_range:
    for W_value in W_range:
        W_set = create_set_W(greedy_list, W_value)
        k = len(W_set)
        non_winners = C - W_set

        # Check for JR
        result1, duration1 = JR(non_winners, approval_dict, W_set, k, n)

        # Check for EJR
        result2, duration2 = EJR(approval_dict, W_set, k, n, C, l)

        results.append((W_value, result1, duration1, result2, duration2))

        results_df = pd.DataFrame(results, columns=['W', 'JR', 'JR Duration', 'EJR', 'EJR Duration'])

        print(tabulate(results_df, headers='keys', tablefmt='pretty'))


+---+---+---------------+----------------------+---------------+----------------------+
|   | W |      JR       |     JR Duration      |      EJR      |     EJR Duration     |
+---+---+---------------+----------------------+---------------+----------------------+
| 0 | 1 | Satisfies JR. | 0.006173849105834961 | Satisfies EJR | 0.022131681442260742 |
+---+---+---------------+----------------------+---------------+----------------------+
+---+---+---------------+----------------------+---------------+----------------------+
|   | W |      JR       |     JR Duration      |      EJR      |     EJR Duration     |
+---+---+---------------+----------------------+---------------+----------------------+
| 0 | 1 | Satisfies JR. | 0.006173849105834961 | Satisfies EJR | 0.022131681442260742 |
| 1 | 2 | Satisfies JR. | 0.004986286163330078 | Satisfies EJR | 0.021940946578979492 |
+---+---+---------------+----------------------+---------------+----------------------+
+---+---+---------------+-------